In [ ]:
import requests
import re
import os
import codecs
import csv
wordcount = 0


# скачивает список авторов
def authors():
    author_tally = []
    for a in range(1, 4):
        where = 'https://ivgazeta.ru/authors/index?Author_page=%d&ajax=yw0&authorPage=3&url_id=21' % (a)
        plain_page = requests.get(where)
        author = re.findall('(/authors/\d+)', plain_page.text)
        for i in author:
            author_tally.append(i)
    print(author_tally)
    return author_tally


# скачивает статьи авторов (на сайте все статьи рассортированы по авторам)
def articles(author_tally):
    article_tally = []
    for i in author_tally:
        where = 'https://ivgazeta.ru' + i
        plain_page = requests.get(where)
        article = re.findall('(read/\d+)', plain_page.text)
        for i in article:
            article_tally.append(i)
    print(article_tally)
    return set(article_tally)


# определяет дату написания статьи - на сайте месяцы записаны словами
def date_recognizer(date):
    calendar = {'января': 1, 'февраля': 2, 'марта': 3, 'апреля': 4, 'мая': 5, 'июня': 6, 'июля': 7, 'августа': 8,
                'сентября': 9, 'октября': 10, 'ноября': 11, 'декабря': 12}
    if len(date) < 3:
        year = 2018
    else:
        year = date[2]
    month = calendar[date[1]]
    date = (year, month)
    return date


# извлекает мета-инфу и чистит текст от хтмл
def text_cleaner_metadata_extractor(i):
    where = 'https://ivgazeta.ru/' + i
    plain_page = requests.get(where)
    raw_data = re.compile('"title">(.*)<h2>Поделиться</h2>', flags=re.DOTALL)
    raw_text = raw_data.findall(plain_page.text)
    raw_text = ''.join(raw_text)


    author_name = re.compile('<div\sclass="author"><a href="/authors/\d*">([А-Я-а-я\s]+)</a></div> ')
    article_name = re.compile('([А-Я-а-я-ёЁ\s,.:!?\d]+)</h1>')
    issue_date = re.compile('<span\sclass="date">([А-Я-а-я,.\d\s]*),')
    topic = re.compile('<span\sclass="catalog">([А-Я-а-я\s,.:!?]+)</span>')

    author = ''.join(author_name.findall(raw_text))
    article = ''.join(article_name.findall(raw_text))
    date = ''.join(issue_date.findall(plain_page.text))
    date = date.split()
    day = date[0]
    date = date_recognizer(date)
    topic = ''.join(topic.findall(plain_page.text))

    text = re.sub('<[^>]*>', '', raw_text, flags=re.DOTALL)
    text = re.sub('&\S* ', '', text)
    text = re.sub('&', ' ', text)
    text = re.sub('[A-Z-a-z]*', '', text)

    return author, article, date, day, topic, text


# пишет неразмеченный текст в файл
def plain_text_file_writer(i, file_name):
    where = 'https://ivgazeta.ru/' + i

    data = text_cleaner_metadata_extractor(i)

    author = data[0]
    article = data[1]
    date = data[2]
    day = data[3]
    topic = data[4]
    text = data[5]
    issue_date = str(day) + '.' + str(date[1]) + '.' + str(date[0])
    year = date[0]
    if not os.path.exists(file_name):
        f = codecs.open(file_name, 'a+', 'utf-8')
        f.write('@au ' + author + '\n')
        f.write('@ti ' + article + '\n')
        f.write('@da ' + issue_date + '\n')
        f.write('@topic ' + topic + '\n')
        f.write('@url ' + where + '\n')
        f.write('\n')
        f.write(text)
        f.close()


# создает таблицу и подписывает колонки
def csv_table_creator():
    mdir = 'Ivgazeta'
    if not os.path.exists(mdir):
        os.makedirs(mdir)
    with open(mdir + '/' + 'metadata.csv', mode='a+', encoding="utf-8") as csv_file:
        fieldnames = ['path', 'author', 'header', 'created', 'sphere', 'topic', 'style',
                      'audience_age', 'audience_level', 'audience_size', 'source',
                      'publication', 'publ_year', 'medium', 'country', 'region', 'language']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames, delimiter='\t')
        writer.writeheader()


# заполняет таблицу
def csv_table_writer(i, file_name):
    where = 'https://ivgazeta.ru/' + i
    data = text_cleaner_metadata_extractor(i)

    author = data[0]
    article = data[1]
    date = data[2]
    day = data[3]
    topic = data[4]
    issue_date = str(day) + '.' + str(date[1]) + '.' + str(date[0])
    year = date[0]

    with open('Ivgazeta' + '/' + 'metadata.csv', mode='a+', encoding="utf-8") as csv_file:
        fieldnames = ['path', 'author', 'header', 'created', 'sphere', 'topic', 'style',
                      'audience_age', 'audience_level', 'audience_size', 'source',
                      'publication', 'publ_year', 'medium', 'country', 'region', 'language']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames, delimiter='\t')

        writer.writerow({'path': file_name, 'author': author, 'header': article, 'created': issue_date,
                         'sphere': 'публицистика', 'topic': topic, 'style': 'нейтральный', 'audience_age': 'н-возраст',
                         'audience_level': 'н-уровень', 'audience_size': 'городская', 'source': where,
                         'publication': 'Ивановская Газета', 'publ_year': year, 'medium': 'газета', 'country': 'Россия',
                         'region': 'Ивановская область', 'language': 'ru'})


# определяет в какую папку положить статью
# pdir - main directory, mydir - mystem directories
# скачивает не все тексты, а столько, чтобы хватило на 100 000 слов
def classifier(article_tally, wordcount):
    pdir = 'Ivgazeta' + '/' + 'plain'
    mydir_xml = 'Ivgazeta' + '/' + 'mystem-xml'
    mydir_plain = 'Ivgazeta' + '/' + 'mystem-plain'
    if not os.path.exists(pdir):
        os.makedirs(pdir)
    if not os.path.exists(mydir_xml):
        os.makedirs(mydir_xml)
    if not os.path.exists(mydir_plain):
        os.makedirs(mydir_plain)

    csv_table_creator()

    article_num = 0
    for i in article_tally:
        data = text_cleaner_metadata_extractor(i)
        date = data[2]
        text = data[5]


        text_wordcount = len(text.split())
        article_num += 1
        wordcount += text_wordcount

        print(wordcount)
        print(article_num, '/', len(article_tally))

        dirname = pdir + '/' + str(date[0]) + '/' + str(date[1])
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        file_name = dirname + '/' + 'article' + str(article_num) + '.txt'
        plain_text_file_writer(i, file_name)
        csv_table_writer(i, file_name)

        if wordcount >= 100000:
            print('enuf')
            break
    return pdir


def main():
    author_tally = authors()
    article_tally = articles(author_tally)
    classifier(article_tally, wordcount)


if __name__ == '__main__':
    main()

